## Hands-on 5 - Gemiddelde sprekertijd berekenen
Nu berekenen we de gemiddelde sprekertijd per geslacht. En daarmee beantwoorden we onze vraag!

In [ ]:
import csv
def load_speaker_data(filename):
    """Leest de spreker data in van de file"""
    df = pd.read_csv(filename)
    return df

In [ ]:
import pandas as pd
import requests
import urllib
def get_gender(name):
    """Gegeven een naam, haalt de geslacht van die persoon op uit Wikidata.
    Geeft een lege waarde terug als geen gender is gevonden"""
    gender = "Unknown"
    query = "SELECT DISTINCT ?genderLabel WHERE {?person wdt:P31 wd:Q5 . ?person ?label \"%s\" .?person wdt:P21 ?gender .SERVICE wikibase:label {  bd:serviceParam wikibase:language \"nl\".}}"%name
    headers = {"Accept":"application/json",
               
               "User-Agent":"wikidataDataAlsKansDemoBot/0.0 (http://www.beeldengeluid.nl)"
              }
    response = requests.get("https://query.wikidata.org/sparql?query=" + urllib.parse.quote(query), headers=headers)
    
    if response.status_code == 200:
        genders = [binding["genderLabel"]["value"] for binding in response.json()["results"]["bindings"] if "genderLabel" in binding]
        if genders:
            gender = genders[0]
    return gender

In [ ]:
def add_gender_for_all_speakers(spreker_data):
    gender_column = []
    for person_name in spreker_data['Naam']:
        print(person_name)
        gender_column.append(get_gender(person_name))        
        time.sleep(3)
        
    spreker_data['Geslacht'] = gender_column
    
    return spreker_data
        

In [ ]:
def calculate_average_per_gender(speaker_data_with_gender):
    pivoted_frame =  pd.pivot_table(speaker_data_with_gender, values=['Spreektijd'], index='Geslacht', aggfunc='mean')
    pivoted_frame = pivoted_frame.reset_index()
    
    return pivoted_frame

In [ ]:
def plot_bar_chart(data, x_axis, y_axis):
    import plotly.express as px

    fig = px.bar(data, x = x_axis, y = y_axis)
    fig.show()

## Spreker data combineren met geslacht

In [ ]:
spreker_data = load_speaker_data("speaker_times.csv")
print(spreker_data)
spreker_geslacht_data = add_gender_for_all_speakers(spreker_data)
print(spreker_geslacht_data)

In [ ]:
average_per_gender = calculate_average_per_gender(spreker_geslacht_data)
print(average_per_gender)

### Plot the results

In [ ]:
plot_bar_chart(average_per_gender, 'Geslacht', 'Spreektijd')